<a href="https://colab.research.google.com/github/shivamVR46/LLM/blob/master/LLM_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install ipdb

In [3]:
import os, sys
import ipdb # for debugging
from tqdm import tqdm
from datetime import datetime
import platform, shutil
import requests, zipfile, io

#Pytorch
import torch
import torch.nn as nn
from torch.nn import functional as F

#tokenizer
import sentencepiece as spm

# For better performance
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cuda.allow_tf32 = True

#Empty GPU memory
torch.cuda.empty_cache()

In [8]:
pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


In [4]:
# ARCHITECTURE PARAMETERS
batch_size = 16  # batch size is nothing but data to be divided into batches so that LLM can learn to guess the next sequence of tokens
                 # required GPU as it can process these batches parallely; size 8 to 128 is enough for 4GB GPU
context = 512    # window (range) that the LLM is going to pay attention to as each input is a token transformed to numbers and tokenizer will
                 # have all the possibibilities of token. Paying attention to larger window is good as LLM cab build relationship between tokens but that requires more memory.
embed_size = 384 # tokens are transformed to numbers and just by using number LLM cannot know what does it means so tokens are tranformed to embedding,
                 # which would be a vector of 384 numbers here where each number is a feature to describe the token.
n_layers = 7     #transformer architecture is of number of layers; each layer has 2 things : attention mechanism (communication) and feed forward layer (computation)
n_head = 7       # In each attention mechanism, there are number of heads so 384 features of each token splitted into 7 heads and then bring them together then to feed forward layer.
BIAS = True

# HYPERPARAMETERS

lr = 3e-4 # learning rate
dropout = 0.05 # prevents overfitting, drop 5% of neurons so that network doesnt memorize everything as it needs to learn to generalize
weight_decay = 0.01 # limit the scale of the parameters of the network, so that it doesnt adapt too tightly to the training dataset
grad_clip = 1.0 # To prevent gradient overflowing or vanishing issue, we need to provide a range for that

# TRAINING PARAMETERS

train_iters = 100000 # Number of top level of training iterations
eval_interval = 50   # We dont use all training data for training ( to prevent overfitting ), thus take percentage of training data for training and for evaluation the other.
eval_iters = 10      # Will make an average over multiple batches rather than just calculate the loss over single batch to get performance data
compile = False      # Way to work with Pytorch; where pytorch will make models runs smoother - requires GPUs
checkpoint_dir = 'models/'  # Folder where the snapshots of the training process would be stored
checkpoint_fn = 'latest.pt' # How to call file that will save the values of the parameters
checkpoint_load_fn = 'latest.pt'  # To restart the training from one of the checkpoints so to take the file storing that.
dtype = torch.bfloat16     # data type (all the numbers/weights/parameters are numbers with different precision) representation of the numbers, floating 16 bit used

# MODE
inference = False  # During training, the weights (parameters) would be tweaked until it learns to predict the next token/word from a sequence of words.
                   # During inference, provides a new set of input of sequence of words and ask to for an answer. Thus during training this is false else true.

# DEVICE
device = "cuda" if torch.cuda.is_available() else "cpu"   # Is to know if GPU or not
print("device: You will be using: ", device)


device: You will be using:  cuda


In [9]:
# LOGGING

wandb_log = True  # wandb.ai which provides free statictics of the model
wandb_project = "llm1"
wandb_run_name = "llm1-"+ datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

if wandb_log:
  import wandb
  wandb.init(project=wandb_project, name=wandb_run_name)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 1228, in init
    wi.setup(kwargs)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 295, in setup
    wandb_login._login(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_login.py", line 346, in _login
    wlogin.prompt_api_key()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_login.py", line 273, in prompt_api_key
    key, status = self._prompt_api_key()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_login.py", line 252, in _prompt_api_key
    key = apikey.prompt_api_key(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/lib/apikey.py", line 164, in prompt_api_key
    key = input_callback(api_ask).strip()
  File "/usr/local/lib/python3.10/dist-packages/click/termui.py", line 164, in prompt
    value = prompt_func(prompt)
  File "/usr/local/lib/python3.10/dist-packages/click/termui.py", li


KeyboardInterrupt



In [11]:
with open('wiki.txt', 'r', encoding='utf-8') as f:
  text = f.read()

print(text[30000:30300])

terms.
For example, there are objects in two groups (as shown on the right). The objects are various shapes, where one group has 3 of them while the other has 2. When the two groups combine into one, the overall amount (sum) of the shapes become 5.

Vertical Addition

The animation above demonstrate
